In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import time
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


In [ ]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [ ]:
#Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [ ]:
#Load the data and extract features for each sound file
def load_data(test_size = 0.2):
  x, y = [], []
  for folder in glob.glob('/content/drive/My Drive/speech-emotion-recognition-ravdess-data/Actor_*'):
    print(folder)
    for file in glob.glob(folder + '/*.wav'):
      file_name = os.path.basename(file)
      emotion = emotions[file_name.split('-')[2]]
      if emotion not in observed_emotions:
        continue
      feature = extract_feature(file, mfcc = True, chroma = True, mel = True)
      x.append(feature)
      y.append(emotion)
  return train_test_split(np.array(x), y, test_size = test_size, random_state = 9)

In [ ]:
file="/content/drive/My Drive/speech-emotion-recognition-ravdess-data/Actor_02/03-01-01-01-01-01-02.wav"
feature=extract_feature(file,mfcc=True,chroma=True,mel=True)
#aaa = librosa.load('/content/drive/My Drive/speech-emotion-recognition-ravdess-data/Actor_10/03-01-01-01-01-01-10.wav')
#file="content/drive/My Drive/speech-emotion-recognition-ravdess-data/Actor_10/03-01-01-01-01-01-10.wav"
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)
print(x_train)
print(x_test)
print(y_train)
print(y_test)

/content/drive/My Drive/speech-emotion-recognition-ravdess-data/Actor_18
/content/drive/My Drive/speech-emotion-recognition-ravdess-data/Actor_20
/content/drive/My Drive/speech-emotion-recognition-ravdess-data/Actor_15
/content/drive/My Drive/speech-emotion-recognition-ravdess-data/Actor_17
/content/drive/My Drive/speech-emotion-recognition-ravdess-data/Actor_23
/content/drive/My Drive/speech-emotion-recognition-ravdess-data/Actor_24
/content/drive/My Drive/speech-emotion-recognition-ravdess-data/Actor_16
/content/drive/My Drive/speech-emotion-recognition-ravdess-data/Actor_21
/content/drive/My Drive/speech-emotion-recognition-ravdess-data/Actor_19
/content/drive/My Drive/speech-emotion-recognition-ravdess-data/Actor_22
/content/drive/My Drive/speech-emotion-recognition-ravdess-data/Actor_12
/content/drive/My Drive/speech-emotion-recognition-ravdess-data/Actor_11
/content/drive/My Drive/speech-emotion-recognition-ravdess-data/Actor_06
/content/drive/My Drive/speech-emotion-recognition-

In [ ]:
#Initialize the multi layer perceptron classifier

model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [ ]:
#Train the model

model.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [ ]:
y_pred=model.predict(x_test)
y_pre=model.predict([feature])
print("The outcome is : ",y_pre)

The outcome is :  ['calm']


In [ ]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 90.94%
